In [22]:
#import packages and set wd to sportfishingreport_project folder
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

os.chdir("C:/Users/Kevin/Documents/Projects/Portfolio/sportfishingreport_project")

In [5]:
#read in data
df = pd.read_csv("sfr_data_cleaned.csv")

In [13]:
#overview of first 3 columns
df.head(3)

,date,location,boat_name,trip_type,num_anglers,catch,date2,anglers,released,restrictions,catch_count,catch_species
0,2020-01-01,central coast,new horizon,full day trip,12 Anglers,"23 Dungeness Crab, 215 Sanddab",2020-01-01,12,0,NaN,23.0,dungeness crab
1,2020-01-01,central coast,new horizon,full day trip,12 Anglers,"23 Dungeness Crab, 215 Sanddab",2020-01-01,12,0,NaN,215.0,sanddab
2,2020-01-01,los angeles,native sun,3/4 day trip,21 Anglers,"25 Sand Bass, 65 Blacksmith, 1 Halibut, 15 Scu...",2020-01-01,21,0,NaN,25.0,sand bass


In [11]:
#column names
df.columns

<bound method Index.sort_values of Index(['date', 'location', 'boat_name', 'trip_type', 'num_anglers', 'catch',
       'date2', 'anglers', 'released', 'restrictions', 'catch_count',
       'catch_species'],
      dtype='object')>

In [21]:
#column datatypes
df.dtypes


True

In [ ]:
#remove redundant columns like 'date' 

In [12]:
#descriptive stats of numerical columns
df.describe()

,anglers,released,catch_count
count,62369.000000,62369.000000,62113.000000
mean,21.006253,0.043499,40.636920
std,10.140372,0.203980,96.345954
min,0.000000,0.000000,0.000000
25%,14.000000,0.000000,3.000000
50%,20.000000,0.000000,12.000000
75%,27.000000,0.000000,46.000000
max,190.000000,1.000000,9500.000000


In [14]:
#look at categorical columns to see unique values and # instances
df.location.value_counts()

san diego           18652
los angeles         11346
ventura coast       10081
orange               9680
central coast        6397
bay area             5259
washington coast      468
north coast           208
oregon coast          198
lake tahoe             39
delta                  38
puget sound             2
florida south           1
Name: location, dtype: int64

In [ ]:
df.